In [1]:
! pip install -q pyspark==3.1.3 spark-nlp

In [1]:
%cd C:\Users\Jasmi\nlp_course\HS_651

C:\Users\Jasmi\nlp_course\HS_651


In [2]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import pandas as pd

In [3]:
from pyspark.ml.feature import SQLTransformer
from pyspark.ml.feature import StringIndexer

In [4]:
import pandas as pd
import numpy as np

In [4]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  3.4.2
Apache Spark version:  3.1.3


In [5]:
readmission=pd.read_csv('df_adm_dis_sum_not_cleaned.csv')

In [7]:
readmission.head(30)

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,...,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,AGE_UPON_ADMIT,READMISSION_TIME,READMISSION_TYPE,READMISSION_INTERVAL,READMISSION_STATUS,TEXT_AGG
0,0,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,...,0,1,M,2025-04-11 00:00:00,77.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2101-10-20**] Discharg...
1,1,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,...,0,1,F,2143-05-12 00:00:00,48.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2191-3-16**] Discharge...
2,2,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,...,0,1,F,2109-06-21 00:00:00,66.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2175-5-30**] Dischar...
3,3,8,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,...,1,1,M,2108-01-26 00:00:00,42.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2149-11-9**] Dischar...
4,4,10,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,...,0,1,F,2128-02-22 00:00:00,50.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2178-4-16**] ...
5,5,11,12,112213,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,...,1,1,M,2032-03-24 00:00:00,72.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2104-8-7**] Discharge ...
6,6,12,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,...,0,1,F,2127-02-27 00:00:00,40.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2167-1-8**] Discharg...
7,7,14,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,...,0,1,F,2087-07-14 00:00:00,47.0,2135-05-09 14:11:00,EMERGENCY,128.0,Non-readmitted,Admission Date: [**2134-12-27**] ...
8,8,15,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,...,0,1,F,2087-07-14 00:00:00,48.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2135-5-9**] D...
9,9,16,18,188822,2167-10-02 11:18:00,2167-10-04 16:15:00,NaN,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,...,0,1,M,2116-11-29 00:00:00,51.0,NaN,NaN,NaN,Non-readmitted,Admission Date: [**2167-9-30**] ...


In [6]:
from sklearn.model_selection import train_test_split
df_train, df_test=train_test_split(readmission,test_size=0.2, random_state=42)

# sub-sampling the negatives (non-readmitted) on the training set
df_train_readm=df_train[df_train.READMISSION_STATUS=='Readmitted']
df_train_non_readm=df_train[df_train.READMISSION_STATUS=='Non-readmitted']
df_train_sub = pd.concat([df_train_readm, df_train_non_readm.sample(n = len(df_train_readm), random_state = 42)],axis = 0)

In [7]:
# Convert the pandas df to a spark df
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
train = spark.createDataFrame(df_train_sub)
test= spark.createDataFrame(df_test)

### Build up Pipelines

In [11]:
%%time
# Produce pipeline for data cleaning and sentence(discharge summary) embedding
document_assembler = DocumentAssembler() \
      .setInputCol("TEXT_AGG") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemmatizer = Lemmatizer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

bert_embeddings = BertEmbeddings.pretrained()\
  .setInputCols(["document","lemma"])\
  .setOutputCol("bert_embeddings")\
  .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert_embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")

label_stringIdx = StringIndexer(inputCol = "READMISSION_STATUS", outputCol = "label")

nlp_pipeline_Bert = Pipeline(
stages=[document_assembler, 
          tokenizer,
            normalizer,
            stopwords_cleaner,
            lemmatizer,
            bert_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            explodeVectors,
            label_stringIdx])

small_bert_L2_768 download started this may take some time.
Approximate size to download 139.6 MB
[OK!]
Wall time: 20.7 s


In [12]:
# nlp_Bert_model= nlp_pipeline_Bert.fit(train)

In [13]:
# nlp_Bert_model.write().overwrite().save('/Models_Pipelines/bert_readmission')

In [8]:
from pyspark.ml.pipeline import PipelineModel
bert_readmission= PipelineModel.load("/Models_Pipelines/bert_readmission")

In [9]:
# Transform training set
processed_bert=bert_readmission.transform(train)

In [17]:
processed_bert.select('TEXT_AGG','features','label').show(truncate=40)

+----------------------------------------+----------------------------------------+-----+
|                                TEXT_AGG|                                features|label|
+----------------------------------------+----------------------------------------+-----+
|Admission Date:  [**2160-4-14**]     ...|[-0.06217242404818535,-0.009191752411...|  1.0|
|Admission Date:  [**2135-4-2**]      ...|[0.005630149971693754,-0.015020967461...|  1.0|
|Admission Date:  [**2150-3-11**]     ...|[0.09284380078315735,0.09849745035171...|  1.0|
|Admission Date:  [**2115-1-26**]     ...|[-0.025607280433177948,-0.06520558893...|  1.0|
|Admission Date:  [**2181-12-13**]    ...|[0.09523189812898636,-0.0035093307960...|  1.0|
|Admission Date:  [**2104-7-15**]     ...|[0.14823265373706818,-0.0947973653674...|  1.0|
|Admission Date:  [**2132-5-15**]     ...|[0.1047961562871933,0.046967465430498...|  1.0|
|Admission Date:  [**2197-3-31**]     ...|[0.009211497381329536,-0.043467480689...|  1.0|
|Admission

In [10]:
# Transform test set
processed_bert_test=bert_readmission.transform(test)

In [11]:
# Convert processed data to pd dataframes
pd_train=processed_bert.select('features','label').toPandas()

C:\Anaconda\envs\nlp_course\lib\site-packages\pyspark\sql\pandas\conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [13]:
pd_test=processed_bert_test.select('features','label').toPandas()

In [12]:
pd_train.to_csv("bert_train.csv")

In [14]:
pd_test.to_csv("bert_test.csv")

In [5]:
pd_train=pd.read_csv("Glove_train.csv")
pd_test=pd.read_csv("glove_test.csv")

In [6]:
pd_train.label=pd_train.label.astype("int")

In [7]:
pd_test.label=pd_test.label.astype("int")

In [8]:
X_train=pd_train.features
y_train=pd_train.label

In [9]:
X_test=pd_test.features
y_test=pd_test.label

In [10]:
# X_test now is lists of strings, which won't be accpetable by machine learning models. The following codes is to tranform it into lists of lists with integers
X_train_trans=[]
for doc in X_train:
    embedding=doc[2:-2]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
        str_to_num_list.append(float(num_str))
    X_train_trans.append(str_to_num_list)

In [11]:
X_test_trans=[]
for doc in X_test:
    embedding=doc[2:-2]
    embedding_list=embedding.split(",")
    str_to_num_list=[]
    for num_str in embedding_list:
        str_to_num_list.append(float(num_str))
    X_test_trans.append(str_to_num_list)

## Modeling

In [60]:
model_bert=[]
Roc_auc_cv=[]
Roc_auc_test=[]

### Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [26]:
# bert vs Logistic regression
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

param= dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

logistic_clf_bert = BayesSearchCV(estimator=LogisticRegression(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [27]:
logistic_clf_bert.fit(X_train_trans,y_train)

C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=LogisticRegression(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'C': [100, 10, 1.0, 0.1, 0.01], 'penalty': ['l2'],
                             'solver': ['newton-cg', 'lbfgs', 'liblinear']})

In [28]:
import pickle
with open('/Models_Pipelines/logistic_clf_bert.pkl','wb') as f:
    pickle.dump(logistic_clf_bert,f)

In [55]:
with open('/Models_Pipelines/logistic_clf_bert.pkl', 'rb') as f:
    logistic_clf_bert = pickle.load(f)

In [56]:
logistic_clf_bert_best=logistic_clf_bert.best_score_

In [57]:
logistic_clf_bert_best

0.6761449608832926

In [58]:
y_prob_logistic_clf_bert = logistic_clf_bert.predict_proba(X_test_trans)
roc_auc_y_prob_logistic_clf_bert=roc_auc_score(y_test, y_prob_logistic_clf_bert[:,1])

In [59]:
roc_auc_y_prob_logistic_clf_bert

0.6870299307179639

In [61]:
model_bert.append("Logistic Regression")
Roc_auc_cv.append(logistic_clf_bert_best)
Roc_auc_test.append(roc_auc_y_prob_logistic_clf_bert)

### Naive Bayes

Naive Bayes doesn't allow negative inputs (there're negative numbers in word embeddings) so I'll skip the combination of GloVe and Naive Bayes

### Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier

max_depth=[2, 3, 5, 10, 20]
min_samples_leaf=[5, 10, 20, 50, 100]
criterion=["gini", "entropy"]

param= dict(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

dec_tree_clf_bert = BayesSearchCV(estimator=DecisionTreeClassifier(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [17]:
dec_tree_clf_bert.fit(X_train_trans,y_train)

C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=DecisionTreeClassifier(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'criterion': ['gini', 'entropy'],
                             'max_depth': [2, 3, 5, 10, 20],
                             'min_samples_leaf': [5, 10, 20, 50, 100]})

In [19]:
import pickle
with open('/Models_Pipelines/dec_tree_clf_bert.pkl','wb') as f:
    pickle.dump(dec_tree_clf_bert,f)

In [20]:
with open('/Models_Pipelines/dec_tree_clf_bert.pkl', 'rb') as f:
    dec_tree_clf_bert = pickle.load(f)

In [21]:
dec_tree_clf_bert_best=dec_tree_clf_bert.best_score_

In [22]:
dec_tree_clf_bert_best

0.6050549554024184

In [23]:
y_prob_dec_tree_clf_bert = dec_tree_clf_bert.predict_proba(X_test_trans)
roc_auc_dec_tree_clf_bert=roc_auc_score(y_test, y_prob_dec_tree_clf_bert[:,1])

In [24]:
roc_auc_dec_tree_clf_bert

0.6152074618180221

In [62]:
model_bert.append("Decision Tree")
Roc_auc_cv.append(dec_tree_clf_bert_best)
Roc_auc_test.append(roc_auc_dec_tree_clf_bert)

### Linear SVM

In [27]:
c_values=[100, 10, 1.0, 0.1, 0.01]

param= dict(C=c_values)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

lsvc_clf_bert = BayesSearchCV(estimator=LinearSVC(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [28]:
lsvc_clf_bert.fit(X_train_trans,y_train)

C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=LinearSVC(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'C': [100, 10, 1.0, 0.1, 0.01]})

In [29]:
with open('/Models_Pipelines/lsvc_clf_bert.pkl','wb') as f:
    pickle.dump(lsvc_clf_bert,f)

In [30]:
with open('/Models_Pipelines/lsvc_clf_bert.pkl', 'rb') as f:
    lsvc_clf_bert = pickle.load(f)

In [33]:
lsvc_clf_bert_best=lsvc_clf_bert.best_score_

In [34]:
lsvc_clf_bert_best

0.676189638133792

In [35]:
y_dec_func_lsvc_clf_bert=lsvc_clf_bert.decision_function(X_test_trans)
roc_auc_lsvc_clf_bert=roc_auc_score(y_test, y_dec_func_lsvc_clf_bert)

In [36]:
roc_auc_lsvc_clf_bert

0.6872842288863896

In [63]:
model_bert.append("Linear SVM")
Roc_auc_cv.append(lsvc_clf_bert_best)
Roc_auc_test.append(roc_auc_lsvc_clf_bert)

### KNN

In [39]:
k_range = list(range(1, 31))
param= dict(n_neighbors=k_range)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

knn_clf_bert = BayesSearchCV(estimator=KNeighborsClassifier(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [40]:
knn_clf_bert.fit(X_train_trans,y_train)

C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=KNeighborsClassifier(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                             12, 13, 14, 15, 16, 17, 18, 19, 20,
                                             21, 22, 23, 24, 25, 26, 27, 28, 29,
                                             30]})

In [41]:
with open('/Models_Pipelines/knn_clf_bert.pkl','wb') as f:
    pickle.dump(knn_clf_bert,f)

In [42]:
with open('/Models_Pipelines/knn_clf_bert.pkl', 'rb') as f:
    knn_clf_bert = pickle.load(f)

In [43]:
knn_clf_bert_best=knn_clf_bert.best_score_

In [44]:
knn_clf_bert_best

0.6379218287933012

In [45]:
y_prob_knn_clf_bert = knn_clf_bert.predict_proba(X_test_trans)
roc_auc_knn_clf_bert=roc_auc_score(y_test,y_prob_knn_clf_bert[:,1])

In [46]:
roc_auc_knn_clf_bert

0.6693182691568673

In [64]:
model_bert.append("KNN")
Roc_auc_cv.append(knn_clf_bert_best)
Roc_auc_test.append(roc_auc_knn_clf_bert)

### Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier
max_depth=[5, 10, 15, 20]
min_samples_leaf=[5, 10, 20, 50, 100]
criterion=["gini", "entropy"]
n_estimators=[10,50,100,150]

param= dict(max_depth=max_depth,min_samples_leaf=min_samples_leaf,criterion=criterion,n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
scoring='roc_auc'

rand_for_clf_bert = BayesSearchCV(estimator=RandomForestClassifier(), search_spaces=param, scoring=scoring, n_jobs=-1, cv=cv)

In [48]:
rand_for_clf_bert.fit(X_train_trans,y_train)

C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\Anaconda\envs\nlp_course\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=1),
              estimator=RandomForestClassifier(), n_jobs=-1, scoring='roc_auc',
              search_spaces={'criterion': ['gini', 'entropy'],
                             'max_depth': [5, 10, 15, 20],
                             'min_samples_leaf': [5, 10, 20, 50, 100],
                             'n_estimators': [10, 50, 100, 150]})

In [49]:
with open('/Models_Pipelines/rand_for_clf_bert.pkl','wb') as f:
    pickle.dump(rand_for_clf_bert,f)

In [50]:
with open('/Models_Pipelines/rand_for_clf_bert.pkl', 'rb') as f:
    rand_for_clf_bert = pickle.load(f)

In [51]:
rand_for_clf_bert_best=rand_for_clf_bert.best_score_

In [52]:
rand_for_clf_bert_best

0.6746394679938784

In [53]:
y_prob_rand_for_clf_bert = rand_for_clf_bert.predict_proba(X_test_trans)
roc_auc_rand_for_clf_bert=roc_auc_score(y_test,y_prob_rand_for_clf_bert[:,1])

In [54]:
roc_auc_rand_for_clf_bert

0.6945667245454787

In [65]:
model_bert.append("Random Forest")
Roc_auc_cv.append(rand_for_clf_bert_best)
Roc_auc_test.append(roc_auc_rand_for_clf_bert)

In [66]:
result_bert=pd.DataFrame({'model_bert': model_bert, 'Roc_auc_cross_val': Roc_auc_cv,'Roc_auc_test':Roc_auc_test})
result_bert=result_bert.sort_values('Roc_auc_test')
result_bert.reset_index(drop = True)

,model_bert,Roc_auc_cross_val,Roc_auc_test
0,Decision Tree,0.605055,0.615207
1,KNN,0.637922,0.669318
2,Logistic Regression,0.676145,0.687030
3,Linear SVM,0.676190,0.687284
4,Random Forest,0.674639,0.694567
